In [19]:
from bs4 import BeautifulSoup
import urllib.request
import requests
from pprint import pprint
import pandas as pd
import json
import copy
from datetime import datetime, timezone
from google.cloud import storage
import gcsfs
import re
from typing import List

In [20]:
storage_client = storage.Client(project="ytone-430507")
today = datetime.now()
api_name = "douyin_user_profile"

In [21]:
processing_blobs = [
{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[1], "%Y-%m-%d"),
    "batch_number": blob.name.split('/')[-1].replace(".json", "").split("_")[-1],
} for blob in storage_client.list_blobs("0_raw_data",prefix="3_douyin/") if api_name in blob.name]
processing_blobs

[{'blob': <Blob: 0_raw_data, 3_douyin/2024-10-13/douyin_user_profile_1014_0.json, 1728841577274902>,
  'date': datetime.datetime(2024, 10, 13, 0, 0),
  'batch_number': '0'}]

In [22]:
processed_blobs = [{
    "blob": blob,
    "date": datetime.strptime(blob.name.split('/')[2], "%Y-%m-%d"),
    "batch_number": blob.name.split('/')[-1].replace(".parquet", "").split("_")[-1],
} for blob in storage_client.list_blobs("3_staging_area",prefix="2_douyin/") if api_name in blob.name]
processed_blobs

[{'blob': <Blob: 3_staging_area, 2_douyin/douyin_user_profile/2024-10-13/douyin_user_profile_241014_0.parquet, 1728896868164079>,
  'date': datetime.datetime(2024, 10, 13, 0, 0),
  'batch_number': '0'}]

In [23]:
to_process = []
for processing_blob in processing_blobs:
    if processing_blob["date"] >= datetime(2024, 9, 17):
        if processing_blob["batch_number"] not in [processed_blob["batch_number"] for processed_blob in processed_blobs if processing_blob["date"] == processed_blob["date"]]:
            to_process.append(processing_blob)
pprint(to_process)
print(len(to_process))

[]
0


In [16]:
def get_douyin_user_profile_data(raw_data):
    result = []
    for data in raw_data:
        record = {}
        user_profile_data = data.get("user_profile_data")
        if user_profile_data:
            user = user_profile_data.get("user")
            if user:
                record["profile_id"] = data["profile_id"]
                record["follower_count"] = user.get("follower_count")
                record["scrape_date"] = str(today.date())
                record["unique_id"] = user.get("unique_id")
                result.append(record)
    return result

In [17]:
for item in to_process:
    raw_data = json.loads(item["blob"].download_as_string())
    result = get_douyin_user_profile_data(raw_data)
    df = pd.DataFrame(result)
    df.to_parquet("gs://3_staging_area/2_douyin/douyin_user_profile/" + str(item["date"].date()) + "/douyin_user_profile_" + today.strftime("%y%m%d") + "_" + str(item["batch_number"]) + ".parquet")